<a href="https://colab.research.google.com/github/lkarjun/reverse-image-search/blob/main/rimg_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages

In [1]:
!pip install fastai -Uqq

     |████████████████████████████████| 197 kB 5.1 MB/s 
     |████████████████████████████████| 60 kB 8.3 MB/s 


In [2]:
from fastai.vision.all import *

dataset_path = Path("../input/image-classification/images/images")

## Here we go

### Dataloader

In [ ]:
list_of_classes = ['architecure', 'art and culture']

fnames = get_image_files(dataset_path/list_of_classes[0])[:300] + get_image_files(dataset_path/list_of_classes[1])[:300]

In [ ]:
item_tfms = Resize(200)
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=64

In [ ]:
dls = ImageDataLoaders.from_path_func(dataset_path, fnames, 
                                lambda x: x.parent.name,
                                batch_tfms=batch_tfms, 
                                item_tfms=item_tfms, 
                                bs=bs)

dls.show_batch(max_n = 3)

In [ ]:
x, y = dls.one_batch()
x.shape, y.shape

### Model

In [ ]:
body = create_body(resnet34, cut=-2)

In [ ]:
head = create_head(nf = 512, n_out = 2)

In [ ]:
class RImgModel(Module):
    def __init__(self, body,head):
        self.body = body
        self.head = head
    
    def forward(self, x):
        x = self.body(x)
        x = self.head(x)
        return self.head(x)

In [ ]:
m

In [ ]:
model = RImgModel(body, head).cuda()

In [ ]:
sample = torch.randn(3, 400, 400).cuda()

In [ ]:
model(sample).shape

In [ ]:
def rimg_splitter(model):
    return [params(model.encoder), params(model.feature_predict_head), params(model.head)]

In [ ]:
learn = Learner(dls, model, splitter=rimg_splitter, metrics=accuracy)

In [ ]:
learn.freeze()

In [ ]:
learn.fit_one_cycle(4, 3e-3)